# Set up environment for GPU (!important)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import keras
import tensorflow as tf
print("TF version", tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

if len(physical_devices) > 0:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)
# If this last line give an error, stop the notebook kernel, reset it and run again

# Train module

## Import required modules to process data

In [ ]:
from keras_preprocessing.image import ImageDataGenerator
import pandas as pd
import numpy as np

## Train configuration

In [ ]:
IMAGE_SIZE      = 96    # Images are 96x96 px
IMAGE_CHANNELS  = 3     # Images are 3 chanell (RGB)

## Load train data info

In [ ]:
# Function to append image file extension to train img ids
def appendExt(id):
    return id + ".tif"

# Load CSVs
traindf = pd.read_csv("/dataset/train_labels.csv")

# Add extensions to id files
traindf["id"] = traindf["id"].apply(appendExt)

# Labels must be strings
traindf["label"] = traindf["label"].astype(str)

# removing this image because it caused a training error previously
traindf[traindf['id'] != 'dd6dfed324f9fcb6f93f46f32fc800f2ec196be2']

# removing this image because it's black
traindf[traindf['id'] != '9369c7278ec8bcc6c880d99194de09fc2bd4efbe']

## Build image data generator

In [ ]:
datagen = ImageDataGenerator(rescale=1./255., validation_split=0.25)

train_generator=datagen.flow_from_dataframe(
    dataframe = traindf,
    directory = "/dataset/train/",
    x_col = "id",
    y_col = "label",
    subset = "training",
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = 10,
    shuffle = True,
    class_mode = "binary",
)

valid_generator=datagen.flow_from_dataframe(
    dataframe = traindf,
    directory = "/dataset/train/",
    x_col = "id",
    y_col = "label",
    subset = "validation",
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size = 10,
    shuffle = True,
    class_mode = "binary",
)


In [ ]:
# Calculate class weigths
from sklearn.utils import class_weight 
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(traindf['label']),
    y=traindf['label']
)
class_weights = dict(enumerate(class_weights))
print("Class weights:", class_weights)

## Build example model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [ ]:
kernel_size = (3,3)
pool_size= (2,2)
first_filters = 32
second_filters = 64
third_filters = 128

dropout_conv = 0.3
dropout_dense = 0.3


model = Sequential()
model.add(Conv2D(first_filters, kernel_size, activation = 'relu', input_shape = (96, 96, 3)))
model.add(Conv2D(first_filters, kernel_size, activation = 'relu'))
model.add(Conv2D(first_filters, kernel_size, activation = 'relu'))
model.add(MaxPooling2D(pool_size = pool_size)) 
model.add(Dropout(dropout_conv))

model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(Conv2D(second_filters, kernel_size, activation ='relu'))
model.add(MaxPooling2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(Conv2D(third_filters, kernel_size, activation ='relu'))
model.add(MaxPooling2D(pool_size = pool_size))
model.add(Dropout(dropout_conv))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(dropout_dense))
model.add(Dense(2, activation = "softmax"))


model.compile(
    optimizers.Adam(lr=0.0001), 
    loss='binary_crossentropy', 
    metrics=['accuracy']
)

model.summary()

## Train the example model

In [ ]:
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size

print("STEP_SIZE_TRAIN:", STEP_SIZE_TRAIN)
print("STEP_SIZE_VALID:", STEP_SIZE_VALID)

# Save best model
checkpointPath = "/usr/src/scripts/best-model.h5"
checkpoint = ModelCheckpoint(
    checkpointPath,
    monitor='val_accuracy',
    verbose=1, 
    save_best_only=True,
    mode='max'
)

# Dynamic learning rate
reduce_lr = ReduceLROnPlateau(
    monitor='val_accuracy',
    factor=0.5,
    patience=2, 
    verbose=1,
    mode='max',
    min_lr=0.00001
)
                                                                
callbacks_list = [checkpoint, reduce_lr]

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=STEP_SIZE_TRAIN,
    class_weight=class_weights,
    validation_data=valid_generator,
    validation_steps=STEP_SIZE_VALID,
    epochs=1, # Only for test!
    verbose=1,
    callbacks=callbacks_list
)


In [ ]:
# model.evaluate(valid_generator, steps=STEP_SIZE_VALID)

## Predict test data

In [ ]:
# # Load test data
# testdf = pd.read_csv("/dataset/sample_submission.csv")
# testdf["id"] = testdf["id"].apply(appendExt)

# # Set up test data generator (only apply normalization)
# test_datagen=ImageDataGenerator(rescale=1./255.)

# test_generator=test_datagen.flow_from_dataframe(
#     dataframe=testdf,
#     directory="/dataset/test/",
#     x_col="id",
#     y_col=None,
#     batch_size=10,
#     shuffle=False,
#     class_mode=None,
#     target_size=(IMAGE_SIZE,IMAGE_SIZE)
# )


# STEP_SIZE_TEST = test_generator.n//test_generator.batch_size


In [ ]:
# test_generator.reset()
# model.predict(test_generator, steps=STEP_SIZE_TEST)